# Pipeline d'Orchestration Multi‑Agents pour la Mise à Jour de Notebook

Ce notebook présente un pipeline 100% Python qui permet de :
1. Exécuter un notebook template via Papermill et y injecter des paramètres.
2. Mettre à jour dynamiquement une cellule ciblée dans le notebook via nbformat.
3. Orchestrer une conversation collaborative entre plusieurs agents (Coder, Reviewer, Admin) utilisant Semantic Kernel, chacun disposant d’un kernel et de plugins spécifiques (ici, NotebookControlPlugin) afin d'interroger l'état du notebook et de soumettre sa validation.

Le workflow itératif permet au Coder Agent de proposer des modifications, au Reviewer Agent d'exécuter et valider le notebook, et à l'Admin Agent de soumettre le tout pour approbation finale.

In [ ]:
# Bloc 1 : Installation et imports (à exécuter une seule fois)
%pip install --quiet papermill nbformat semantic-kernel

import asyncio
import logging
import nbformat
import papermill as pm
from datetime import datetime 

# Imports de Semantic Kernel
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import ChatHistory, AuthorRole, ChatMessageContent
from semantic_kernel.functions import KernelArguments, kernel_function, KernelFunctionFromPrompt
from semantic_kernel.agents.strategies import KernelFunctionTerminationStrategy, KernelFunctionSelectionStrategy

# Configuration des logs avec formatage détaillé
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[logging.StreamHandler()]
)
# Remplacer la configuration des logs existante par :
class ColorFormatter(logging.Formatter):
    colors = {
        'DEBUG': '\033[94m',
        'INFO': '\033[92m',
        'WARNING': '\033[93m',
        'ERROR': '\033[91m',
        'CRITICAL': '\033[91m\033[1m'
    }
    reset = '\033[0m'

    def format(self, record):
        color = self.colors.get(record.levelname, '')
        message = super().format(record)
        return f"{color}{message}{self.reset}" if color else message

logger = logging.getLogger('Orchestration')
logger.setLevel(logging.INFO)

handler = logging.StreamHandler()
handler.setFormatter(ColorFormatter(
    '%(asctime)s [%(levelname)s] %(name)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
))
logger.addHandler(handler)

# Désactiver les logs verbeux des librairies
logging.getLogger("papermill").setLevel(logging.WARNING)
logging.getLogger("semantic_kernel").setLevel(logging.WARNING)
logging.getLogger("openai").setLevel(logging.WARNING)


print("Packages installés et imports effectués.")


## Partie 1 : Exécution paramétrée et mise à jour du Notebook

1. Nous chargeons un notebook template (Notebook-Template.ipynb) et y injectons l'instruction de tâche via Papermill.
2. Ensuite, nous mettons à jour une cellule ciblée (repérant le marqueur unique TASK_INSTRUCTION_PLACEHOLDER) via nbformat, puis nous réexécutons le notebook pour obtenir la version finale.

In [ ]:
# Exécution du notebook template avec Papermill
source_notebook = "Notebook-Template.ipynb" 
output_notebook = "Notebook-Updated.ipynb"
final_notebook = "Notebook-Final.ipynb"

task_instruction = "Créer un notebook Python qui requête DBpedia via SPARQL et affiche un graphique Plotly."

logger.info("🚀 Début de l'exécution Papermill avec paramètre: %s", task_instruction)
pm.execute_notebook(
    source_notebook,
    output_notebook,
    parameters=dict(TASK_INSTRUCTION=task_instruction)
)
logger.info("✅ Papermill a généré: %s", output_notebook)
print(f"Notebook exécuté sous '{output_notebook}'.")

# Fonction de mise à jour enrichie avec logging
def update_notebook_cell(notebook_path: str, unique_marker: str, new_content: str):
    try:
        logger.info(f"🔍 Recherche du marqueur '{unique_marker}' dans {notebook_path}")
        nb = nbformat.read(notebook_path, as_version=4)
        logger.info(f"📄 Nombre de cellules initiales : {len(nb.cells)}")
        
        found = False
        for idx, cell in enumerate(nb.cells):
            source = cell.get("source", "")
            if unique_marker in source:
                logger.info(f"🎯 Cellule trouvée à l'index {idx} - Type: {cell.cell_type}")
                logger.info("👉 Avant mise à jour : %s", source[:100].replace("\n", " "))
                cell["source"] = new_content
                nbformat.write(nb, notebook_path)
                logger.info("✅ Cellule %d mise à jour avec succès", idx)
                logger.info("👉 Après mise à jour : %s", cell["source"][:100].replace("\n", " ") + ("..." if len(cell["source"]) > 100 else ""))
                found = True
                break
        
        if not found:
            error_msg = f"❌ Marqueur '{unique_marker}' non trouvé parmi {len(nb.cells)} cellules"
            logger.error(error_msg)
            raise ValueError(error_msg)
    except Exception as e:
        logger.exception(f"💥 Échec critique lors de la mise à jour du notebook: {str(e)}")
        raise



# Appel avec log du résultat
update_notebook_cell(output_notebook, "TASK_INSTRUCTION_PLACEHOLDER", 
                     f"TASK_INSTRUCTION_PLACEHOLDER\nTask: {task_instruction}\nUpdated by AI Agent")

logger.info("🔄 Réexécution du notebook mis à jour")
pm.execute_notebook(output_notebook, final_notebook)
logger.info("✅ Notebook final généré: Notebook-Final.ipynb")


## Partie 1bis : Visualisation Initiale du Notebook

Avant de lancer la conversation entre agents, affichons une vue structurée du notebook template pour donner le contexte initial :

In [ ]:
# Nouvelle cellule après l'exécution de Papermill (Bloc 2)

from IPython.display import display, HTML, Markdown

# Nouvelle version enrichie de display_notebook
def display_notebook(notebook_path: str, max_cells: int = 5):
    try:
        nb = nbformat.read(notebook_path, as_version=4)
        display(HTML(f"<h3 style='color:#26608e'>📓 Contenu initial de {notebook_path}</h3>"))
        
        for i, cell in enumerate(nb.cells[:max_cells]):
            cell_type = cell.cell_type.capitalize()
            header = f"### Cellule {i} ({cell_type})"
            content = cell.source.replace("\n", "<br>")[:500] + ("..." if len(cell.source) > 500 else "")
            
            # Ajout d'une couleur de bordure selon le type
            border_color = "#e0e0e0" if cell_type == "Code" else "#f0f0f0"
            
            display(HTML(f"""
                <div style='border-left:4px solid {border_color}; padding:10px; margin:15px; border-radius:4px'>
                    <div style='font-family: Monaco, monospace; font-size:0.9em; color:#404040'>{content}</div>
                </div>
            """))
            logger.info(f"📄 Cell {i} preview: {cell.source[:500].strip()}...")
            
    except Exception as e:
        logger.error(f"🚨 Erreur d'affichage du notebook : {str(e)}")
        raise

# Afficher le template initial avant l'exécution
display_notebook(source_notebook)

## Partie 2 : Intégration d'un Plugin de Contrôle du Notebook

Nous définissons un plugin (NotebookControlPlugin) qui expose des fonctions pour :
- Obtenir le statut du notebook (par exemple, le nombre de cellules, les erreurs éventuelles) via `get_notebook_status`.
- Soumettre le notebook pour approbation via `submit_for_approval`.

Ces fonctions permettront aux agents d'interroger l'état du notebook et de déclencher des actions.

In [11]:
# Classe d'état partagée pour le notebook
class NotebookState:
    def __init__(self, notebook_path: str):
        self.notebook_path = notebook_path
    def __str__(self):
        return f"NotebookState(notebook_path='{self.notebook_path}')"

shared_state = NotebookState(final_notebook)

# Plugin pour le Coder Agent
class NotebookControlPluginCoder:
    def __init__(self, state):
        self.state = state
        self.logger = logging.getLogger("CoderPlugin")

    @kernel_function(description="Met à jour une cellule identifiée par un marqueur unique.")
    def update_cell(self, marker: str, new_content: str) -> str:
        self.logger.info("📤 Appel de update_cell avec marker: %s", marker)
        try:
            nb = nbformat.read(self.state.notebook_path, as_version=4)
            self.logger.info("Lecture du notebook '%s'; %d cellules trouvées.", self.state.notebook_path, len(nb.cells))
            found = False
            for cell in nb.cells:
                if marker in cell.get("source", ""):
                    cell["source"] = new_content
                    found = True
                    break
            if not found:
                error_msg = f"❌ Erreur : Le marqueur '{marker}' n'a pas été trouvé."
                self.logger.error(error_msg)
                return error_msg
            nbformat.write(nb, self.state.notebook_path)
            self.logger.info("✅ Mise à jour effectuée. Nouveau contenu (truncated): %s", new_content[:100])
            return f"✅ Cellule mise à jour dans '{self.state.notebook_path}'."
        except Exception as ex:
            error_msg = f"❌ Erreur lors de la mise à jour: {ex}"
            self.logger.error(error_msg)
            return error_msg

    @kernel_function(description="Récupère le contenu actuel d'une cellule spécifique.")
    def get_cell_content(self, cell_index: int) -> str:
        self.logger.info("Lecture de la cellule %d", cell_index)
        nb = nbformat.read(self.state.notebook_path, as_version=4)
        if 0 <= cell_index < len(nb.cells):
            cell = nb.cells[cell_index]
            return f"Cellule {cell_index} ({cell.cell_type}):\n{cell.source[:200]}{'...' if len(cell.source)>200 else ''}"
        return f"Index {cell_index} invalide; le notebook contient {len(nb.cells)} cellules."




class NotebookControlPluginReviewer:
    def __init__(self, state):
        self.state = state
        self.logger = logging.getLogger("ReviewerPlugin")

    @kernel_function(description="Exécute le notebook et retourne un rapport de validation.")
    def get_notebook_status(self) -> str:
        try:
            executed_path = self.state.notebook_path.replace(".ipynb", "-VALIDATED.ipynb")
            self.logger.info("⏳ Exécution de validation du notebook '%s'", self.state.notebook_path)
            pm.execute_notebook(
                self.state.notebook_path,
                executed_path,
                kernel_name="python3",
                progress_bar=False
            )
            nb = nbformat.read(executed_path, as_version=4)
            stats = {"cells": len(nb.cells), "errors": 0, "outputs": 0}
            error_details = []
            for idx, cell in enumerate(nb.cells):
                if cell.cell_type == "code":
                    stats["outputs"] += len(cell.get("outputs", []))
                    for output in cell.get("outputs", []):
                        if output.get("output_type") == "error":
                            stats["errors"] += 1
                            error_details.append(f"🚨 Cellule {idx}: {output.get('ename')} - {output.get('evalue')}")
            report = [
                "📊 RAPPORT DE VALIDATION",
                f"• Cellules: {stats['cells']}",
                f"• Sorties: {stats['outputs']}",
                f"• Erreurs: {stats['errors']}",
            ]
            if error_details:
                report.extend(error_details)
                report.append("❌ ERREURS DÉTECTÉES")
            else:
                report.append("✅ VALIDATION TERMINÉE")
            final_report = "\n".join(report)
            self.logger.info("Rapport de validation généré.")
            return final_report
        except Exception as e:
            self.logger.error("🔥 Échec de la validation: %s", e)
            return f"❌ Validation impossible: {e}"



# Plugin pour l'Admin Agent
class NotebookControlPluginAdmin:
    def __init__(self, state):
        self.state = state
        self.logger = logging.getLogger("AdminPlugin")

    @kernel_function(description="Soumet le notebook pour approbation.")
    def submit_for_approval(self) -> str:
        self.logger.info("📝 Soumission du notebook '%s' pour approbation.", self.state.notebook_path)
        # Ici, vous pouvez ajouter des vérifications supplémentaires si besoin.
        return f"✅ Notebook '{self.state.notebook_path}' soumis et approuvé."

    



## Partie 3 : Création des Agents (Coder, Reviewer, Admin) avec Plugins

Chaque agent aura son propre kernel auquel sera ajouté un plugin spécifique lui permettant d'accéder aux fonctions de contrôle du notebook.
- Le **Coder Agent** utilisera NotebookControlPluginCoder pour éditer les cellules.
- Le **Reviewer Agent** utilisera NotebookControlPluginReviewer pour obtenir le statut du notebook.
- L’**Admin Agent** utilisera NotebookControlPluginAdmin pour soumettre le notebook pour approbation.

In [ ]:
def create_kernel_for_agent(service_id: str, plugin_instance) -> Kernel:
    k = Kernel()
    # Ajout du service OpenAIChatCompletion pour cet agent
    k.add_service(OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-4o-mini"))
    # Ajout du plugin spécifique, si fourni
    if plugin_instance is not None:
        k.add_plugin(plugin_instance, plugin_name="nbcontrol")
    return k

# Instructions et noms pour chaque agent
CODER_AGENT_NAME = "Coder_Agent"
CODER_AGENT_INSTRUCTIONS = "Vous êtes le Coder. Vous devez mettre à jour les cellules de code selon l'instruction fournie."

REVIEWER_AGENT_NAME = "Reviewer_Agent"
REVIEWER_AGENT_INSTRUCTIONS = "Vous êtes le Reviewer. Exécutez le notebook complet, vérifiez son statut via get_notebook_status et fournissez des retours."

ADMIN_AGENT_NAME = "Admin_Agent"
ADMIN_AGENT_INSTRUCTIONS = "Vous êtes l'Admin. Vérifiez que le notebook est correct et soumettez-le pour approbation via submit_for_approval."

# Création des plugins spécifiques pour chaque agent (en partageant l'état)
coder_plugin = NotebookControlPluginCoder(shared_state)
reviewer_plugin = NotebookControlPluginReviewer(shared_state)
admin_plugin = NotebookControlPluginAdmin(shared_state)

# Création des kernels dédiés pour chaque agent
coder_kernel = create_kernel_for_agent(CODER_AGENT_NAME, coder_plugin)
reviewer_kernel = create_kernel_for_agent(REVIEWER_AGENT_NAME, reviewer_plugin)
admin_kernel = create_kernel_for_agent(ADMIN_AGENT_NAME, admin_plugin)

# Création des agents
coder_agent = ChatCompletionAgent(
    service_id=CODER_AGENT_NAME,
    kernel=coder_kernel,
    name=CODER_AGENT_NAME,
    instructions=CODER_AGENT_INSTRUCTIONS
)

reviewer_agent = ChatCompletionAgent(
    service_id=REVIEWER_AGENT_NAME,
    kernel=reviewer_kernel,
    name=REVIEWER_AGENT_NAME,
    instructions=REVIEWER_AGENT_INSTRUCTIONS
)

admin_agent = ChatCompletionAgent(
    service_id=ADMIN_AGENT_NAME,
    kernel=admin_kernel,
    name=ADMIN_AGENT_NAME,
    instructions=ADMIN_AGENT_INSTRUCTIONS
)

# Création d'un group chat avec les trois agents
group_chat = AgentGroupChat(agents=[coder_agent, reviewer_agent, admin_agent])
print("Group chat multi-agent créé avec plugins spécifiques (état partagé).")


## Partie 4 : Workflow Itératif Complet

Dans cette section, nous mettons en place un workflow itératif où :
- L'utilisateur envoie une demande de mise à jour.
- Le **Coder Agent** propose des modifications via sa fonction d'édition.
- Le **Reviewer Agent** exécute le notebook et vérifie son statut via `get_notebook_status`.
- L’**Admin Agent** intervient pour soumettre le notebook pour approbation via `submit_for_approval`.

Le passage de main (sélection du prochain agent) et la terminaison de la conversation sont gérés par des stratégies basées sur des fonctions kernel (créées à partir de prompts).

In [13]:
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.agents.strategies import KernelFunctionTerminationStrategy, KernelFunctionSelectionStrategy

# Pour extraire le contenu texte d'un résultat éventuel (si c'est un ChatMessageContent), on définit des parsers dédiés
def selection_result_parser(result):
    value = result.value
    if value is None:
        return CODER_AGENT_NAME
    # Si le résultat est une liste, on prend le contenu du premier élément
    if isinstance(value, list):
        first = value[0]
        if hasattr(first, "content"):
            return first.content.strip()
        else:
            return str(first).strip()
    # Si le résultat possède l'attribut 'content', on le retourne
    if hasattr(value, "content"):
        return value.content.strip()
    return str(value).strip()


def termination_result_parser(result):
    value = result.value
    if value is None:
        return False
    if isinstance(value, list):
        first = value[0]
        text = first.content if hasattr(first, "content") else str(first)
    elif hasattr(value, "content"):
        text = value.content
    else:
        text = str(value)
    return "approved" in text.lower()


# Définition d'une fonction de terminaison via prompt
termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt="""
Examine the conversation history and determine if the notebook is approved.
If the notebook is approved, respond with a single word: "approved".
Otherwise, respond with "continue".
History:
{{$history}}
"""
)

# Définition d'une fonction de sélection via prompt
selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
Determine which agent should take the next turn based on the conversation history.
Rules:
- After user input, it is the Coder_Agent's turn.
- After Coder_Agent responds, it is Reviewer_Agent's turn.
- After Reviewer_Agent responds, it is the Coder_Agent's turn again.
- Admin_Agent intervenes only when the Reviewer indicates readiness for approval.
History:
{{{{$history}}}}
Return only the agent name.
"""
)

# Création d'une instance de kernel pour les stratégies (pour la sélection et la terminaison)
def create_kernel_for_strategy(service_id: str) -> Kernel:
    k = Kernel()
    k.add_service(OpenAIChatCompletion(service_id=service_id))
    return k

termination_strategy = KernelFunctionTerminationStrategy(
    agents=[admin_agent],
    function=termination_function,
    kernel=create_kernel_for_strategy("termination"),
    result_parser=termination_result_parser,  # Utilisation de la fonction de parsing corrigée
    history_variable_name="history",
    maximum_iterations=10
)


selection_strategy = KernelFunctionSelectionStrategy(
    function=selection_function,
    kernel=create_kernel_for_strategy("selection"),
    result_parser=selection_result_parser,  # Utilisation de la fonction de parsing corrigée
    agent_variable_name="_agent_",
    history_variable_name="history"
)


# Création d'un nouveau group chat itératif intégrant ces stratégies
iterative_group_chat = AgentGroupChat(
    agents=[coder_agent, reviewer_agent, admin_agent],
    termination_strategy=termination_strategy,
    selection_strategy=selection_strategy
)




In [ ]:
async def run_iterative_workflow():
    chat_history = ChatHistory()
    # Injection du contexte initial avec un extrait du template
    nb_content = "\n".join([c.source[:100] for c in nbformat.read(source_notebook, as_version=4).cells[:3]])
    initial_context = f"""
## CONTEXTE INITIAL ##
Notebook template: {source_notebook}
Extrait de cellules clés:
{nb_content}
"""
    chat_history.add_system_message(initial_context)
    logger.info("📂 Contexte initial injecté : %.80s...", initial_context.replace("\n", " "))

    # Message utilisateur initial
    user_request = "Veuillez implémenter une requête SPARQL sur DBpedia avec visualisation Plotly."
    chat_history.add_user_message(user_request)
    print(f"# User: '{user_request}'")

    iteration = 0
    async for msg in iterative_group_chat.invoke():
        iteration += 1
        # On stocke le numéro de l'itération dans les métadonnées du message (si ce n'est pas déjà fait)
        msg.metadata["iteration"] = iteration
        logger.info("[Tour %d] %s: %.120s...", iteration, msg.role.name, msg.content.replace("\n", " "))
        
        # Toutes les 2 itérations, on affiche l'état actuel du notebook
        if iteration % 2 == 0:
            current_state = nbformat.read(shared_state.notebook_path, as_version=4)
            logger.info("📌 État du notebook '%s': %d cellules", shared_state.notebook_path, len(current_state.cells))
    
    # Affichage du rapport final
    final_report = f"""
=== RAPPORT FINAL ===
🗂 Notebook final : {final_notebook}
🔄 Cellules modifiées : {sum(1 for c in nbformat.read(final_notebook, as_version=4).cells if "Updated by AI" in c.source)}
✅ Dernière exécution : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
📊 Statut : {"APPROUVÉ" if iterative_group_chat.is_complete else "EN ÉCHEC"}
"""
    from IPython.display import Markdown, display
    display(Markdown(final_report))
    logger.info(final_report.replace("\n", " "))
    
    # Si la conversation est terminée, l'Admin soumet le notebook
    submit_func = admin_kernel.get_function("nbcontrol", "submit_for_approval")
    approval_result = await admin_kernel.invoke(submit_func, KernelArguments(notebook_path=shared_state.notebook_path))
    print(f"[Admin] Submission result: {approval_result}")
    logger.info("[Admin] Notebook soumis pour approbation: %s", approval_result)

await run_iterative_workflow()


## Conclusion et Perspectives

Ce notebook intègre désormais un workflow complet et itératif qui combine :
1. L'exécution paramétrée d'un notebook template via Papermill.
2. La mise à jour dynamique d'une cellule ciblée avec nbformat.
3. Une orchestration collaborative multi‑agents (Coder, Reviewer, Admin) avec Semantic Kernel, chacun disposant d'un kernel et de plugins spécifiques pour contrôler l'état du notebook.
4. Des stratégies de sélection et de terminaison (définies via des fonctions kernel) permettant de passer la main de manière itérative jusqu'à ce que le notebook soit approuvé.

Les prochaines étapes pourraient inclure :
- L'enrichissement des plugins pour gérer d'autres aspects (logs, rollback, etc.).
- L'ajustement des stratégies de passage de main pour une sélection plus fine de l'agent suivant.
- La gestion avancée de l'historique de la conversation pour une meilleure réduction des messages.

Bonne exploration !